<a href="https://colab.research.google.com/github/DenizAhmadi/DjangoProject/blob/master/Naive_Bayes_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import re

import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score

import math

import nltk

from sklearn.feature_extraction.text import CountVectorizer

from collections import defaultdict



In [123]:
data = pd.read_csv('train_data.csv')

data

,Unnamed: 0,textID,text,label,label_text
0,14951,7627d7ca56,Hi : Found you through . i am indeed a pigment...,1,neutral
1,6051,7527ed21ee,I`m so unhappy being here. I hate it. It`s aff...,0,negative
2,12712,2f6d9d1419,#stackeoverflow http://bit.ly/13Nfk1,1,neutral
3,2930,9afe83da6a,I`m completely exhausted thanks to yesterday. ...,0,negative
4,2174,322b61740c,"90 degrees, gross skies, and thunderstorms...p...",2,positive
...,...,...,...,...,...
19780,11964,96674184d6,_X sorry about that.,0,negative
19781,21575,14ba250631,@_Bella_Cullen13 posted and its good,2,positive
19782,5390,ae19d98db6,:: holy kraut. I canNOT stay up this late. But...,0,negative
19783,860,e9fdfb3211,with your bicycle?,1,neutral


In [124]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:
data = data.loc[:,['text','label']]
data.drop_duplicates(subset=['text'],inplace=True)
data.dropna(subset=['text','label'],inplace=True)
data.head()

,text,label
0,Hi : Found you through . i am indeed a pigment...,1
1,I`m so unhappy being here. I hate it. It`s aff...,0
2,#stackeoverflow http://bit.ly/13Nfk1,1
3,I`m completely exhausted thanks to yesterday. ...,0
4,"90 degrees, gross skies, and thunderstorms...p...",2


In [126]:
def remove_tags(string):
    removelist = ""
    result = re.sub('','',string)
    result = re.sub("@",'',string)
    result = re.sub("#",'',string)         #remove HTML tags
    result = re.sub('https://.*','',result)   #remove URLs
      #remove non-alphanumeric characters
    result = result.lower()
    return result
data['text']=data['text'].apply(lambda cw : remove_tags(cw))
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [127]:
data

,text,label
0,hi : found . indeed pigment imagination,1
1,i`m unhappy here. hate it. it`s affecting ever...,0
2,stackeoverflow http://bit.ly/13nfk1,1
3,i`m completely exhausted thanks yesterday. go ...,0
4,"90 degrees, gross skies, thunderstorms...perfe...",2
...,...,...
19780,_x sorry that.,0
19781,@_bella_cullen13 posted good,2
19782,:: holy kraut. cannot stay late. book done aut...,0
19783,bicycle?,1


In [128]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [129]:
!python3 -c "import nltk; nltk.download('averaged_perceptron_tagger')"

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [130]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])
data['text'] = data.text.apply(lemmatize_text)

In [131]:
data

,text,label
0,hi : found . indeed pigment imagination,1
1,i`m unhappy here. hate it. it`s affecting ever...,0
2,stackeoverflow http://bit.ly/13nfk1,1
3,i`m completely exhausted thanks yesterday. go ...,0
4,"90 degrees, gross skies, thunderstorms...perfe...",2
...,...,...
19780,_x sorry that.,0
19781,@_bella_cullen13 posted good,2
19782,:: holy kraut. cannot stay late. book done aut...,0
19783,bicycle?,1


In [132]:
text = data['text'].values
label = data['label']

In [133]:
from sklearn.model_selection import train_test_split
train_sentences, test_sentences, train_labels, test_labels = train_test_split(text, label, stratify = label)

In [134]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(max_features=3000)
X = vec.fit_transform(train_sentences)
vocab = vec.get_feature_names_out()  # Use get_feature_names_out() instead of get_feature_names()
X = X.toarray()
unique_labels = set(train_labels)
print("Unique Labels:", unique_labels)

word_counts = {l: defaultdict(lambda: 0) for l in unique_labels}

for i, l in enumerate(train_labels):
    try:
        if l in word_counts:
            for j in range(len(vocab)):
                word_counts[l][vocab[j]] += X[i][j]
        else:
            print(f"Label {l} not found in word_counts dictionary.")
            print("Index:", i)
            print("Full Data Entry:", X[i])
    except KeyError as e:
        print(f"KeyError: {e} at Index {i}")


Unique Labels: {0, 1, 2}


In [135]:
def laplace_smoothing(n_label_items, vocab, word_counts, word, text_label):
    a = word_counts[text_label][word] + 1
    b = n_label_items[text_label] + len(vocab)
    return math.log(a/b)

In [138]:
def group_by_label(x, y, labels):
    data = {}
    for l in labels:
        data[l] = x[np.where(y == l)]
    return data
def fit(x, y, labels):
    n_label_items = {}
    log_label_priors = {}
    n = len(x)
    grouped_data = group_by_label(x, y, labels)
    for l, data in grouped_data.items():
        n_label_items[l] = len(data)
        log_label_priors[l] = math.log(n_label_items[l] / n)
    return n_label_items, log_label_priors


In [139]:
def predict(n_label_items, vocab, word_counts, log_label_priors, labels, x):
    result = []
    for text in x:
        label_scores = {l: log_label_priors[l] for l in labels}
        words = set(w_tokenizer.tokenize(text))
        for word in words:
            if word not in vocab: continue
            for l in labels:
                log_w_given_l = laplace_smoothing(n_label_items, vocab, word_counts, word, l)
                label_scores[l] += log_w_given_l
        result.append(max(label_scores, key=label_scores.get))
    return result

In [140]:
labels = [0,1]
n_label_items, log_label_priors = fit(train_sentences,train_labels,labels)
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred))

Accuracy of prediction on test set :  0.4915082895268904
